Este trabajo ha sido realizado por: Antonio Ramón Vázquez Ramírez para CodeIntelligenceAcademy

# Preajustes y generación de base de datos

Necesitamos crear un modelo de lenguaje que genere transcripciones para la reserva de un vuelo con la información sobre el vuelo (fecha, aeropuerto de salida y llegada, asientos que se van a reservar)

Para esta tarea vamos a necesitar:

* Generar un modelo de lenguaje con las caracteristicas deseadas
* Acceder a los datos de vuelo, en este caso almacenados en Google Drive personal
* Aportar los datos de vuelos como contexto que el modelo pueda utilizar

* la cantidad de vuelos ha tenido que ser reducida. Ya que al generar vuelos para todo un año OpenAI pide añadir un método de pago al tener que tratar con archivos demasiado grandes

## Descarga y ejecución del generador de vuelos

*Descargamos las librerias necesarios

*Descargamos los archivos de generación de datos. Cada vez que se ejecute el cuaderno se generaran nuevos datos

In [1]:
# Librerias de archivos

!pip install gdown --q

# Librerias de IA
!pip install langchain -q
!pip install openai -q
!pip install langchain_openai -q
!pip install beautifulsoup4 -q
!pip install faiss-cpu -q


#Adicion de contexto
!pip install chromadb --q
!pip install sentence-transformers --q

 #modificar numero de tokens
!pip install tiktoken --q

# para generar el pdf con la reserva
!pip install reportlab --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#IMPORTAMOS LIBRERIAS NECESARIAS

#Librerias gestión de archivos y directorios
import os

from reportlab.lib.pagesizes import letter #para generar los pdfs
from reportlab.pdfgen import canvas

#from google.colab import drive, userdata
import gdown
from google.colab import userdata

#Librerias de analis de datos
import pandas as pd
from datetime import datetime


#Librerias de modelos de lenguaje
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, AIMessage, SystemMessage

from langchain.prompts.chat import(
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

from langchain.schema.messages import ChatMessage

from langchain.memory import ChatMessageHistory #Para llevar registros de chat

from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


#Librerias de IA para interpretar documentos
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [3]:
!gdown https://drive.google.com/drive/folders/1jYiiKrb3X-Oj4v1JNCH2L-BYnZVj6185?usp=sharing --folder

Retrieving folder contents
Retrieving folder 1mD6_1hKMQCYmI2z0FMISaI87TSICJdfe common
Retrieving folder 1rdIynMhlmjRIW-AoCnyFp83WqHIOF60L __pycache__
Processing file 1ttVHD2-p3otLQpaIYbU_yXmcKk_U8APu Archivos.cpython-38.pyc
Processing file 1t__n4aLvoxd_oXNgF0tnexbrm8WapLYH Archivos.cpython-311.pyc
Processing file 1tdphPpoldF2TEUbfXQ7v48a_R51Aki2x Clases.cpython-38.pyc
Processing file 1u91dVAGDJyrXrUs7_qRjULCZiB-IubPu Clases.cpython-311.pyc
Processing file 1tiJY-uIaSGndqNvma7jJzRIyv3NUBbFB Funciones_generacion.cpython-38.pyc
Processing file 1tWDEFzVLgqYa8lIpdO1u8HppXYXfVRzF Funciones_generacion.cpython-311.pyc
Processing file 1to4UQhuxkkICrX9qJPgped1sBO_LS9l3 Parametrizacion.cpython-38.pyc
Processing file 1rj79bJpDUtB379anDUSmzWyZ1JHylVrX Archivos.py
Processing file 1rkCbba7Dwy4GkWxDEWao_ZpPtLWwVCSK Clases.py
Processing file 1rgUnJRitu1IZX_FL1-Tyvxy4yWFYawdU Funciones_generacion.py
Processing file 1rVzDAp4Hgm9JNBKXPHV2FIeHrbUyoS5F Parametrizacion.py
Processing file 1n2OXsfJyQj2S7xRzCQ20

In [4]:
os.getcwd()
%cd /content/trabajo_final/

/content/trabajo_final


In [5]:
!python Generador_vuelos_main.py
archivo_path= "vuelos.csv"


El archivo de vuelos ha sido generado 

vuelos.csv  ha sido generado en el directorio del programa python:  /content/trabajo_final


In [6]:
df = pd.read_csv(archivo_path)
df.describe()

,id,plazas_totales,plazas_disponibles,plazas_busisness_totales,plazas_business_disponibles,plazas_turista_totales,plazas_turista_disponibles,precio_business,precio_turista
count,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000
mean,1704.000000,281.013139,102.671533,40.963504,15.218978,240.049635,87.452555,800.314394,388.925869
std,197.886752,67.708837,75.449017,8.053557,10.770815,59.807321,65.214286,300.811407,181.579664
min,1362.000000,180.000000,0.000000,30.000000,0.000000,150.000000,0.000000,330.000000,132.000000
25%,1533.000000,180.000000,29.000000,30.000000,4.000000,150.000000,24.000000,561.040000,245.980000
50%,1704.000000,290.000000,98.000000,40.000000,16.000000,250.000000,82.000000,725.270000,346.080000
75%,1875.000000,348.000000,165.000000,50.000000,24.000000,298.000000,141.000000,1052.880000,483.800000
max,2046.000000,348.000000,289.000000,50.000000,42.000000,298.000000,250.000000,1650.610000,978.610000


# Creación del asistente de reservas de vuelos

Necesitamos crear un modelo de lenguaje que genere transcripciones para la reserva de un vuelo con la información sobre el vuelo (fecha, aeropuerto de salida y llegada, asientos que se van a reservar)

Para esta tarea vamos a necesitar:

* Generar un modelo de lenguaje con las caracteristicas deseadas
* Acceder a los datos de vuelo
* Aportar los datos de vuelos al modelo de lenguaje

In [7]:
key= userdata.get('OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = key

## Generación de la información del vuelo con chat

generamos transcripciones

In [8]:

template_1="""

Actua como un asistente para reservar vuelos con precios estandarizados.

Debes crear una transcripción de chat donde un cliente de nombre {cliente} trata
de reservar unos vuelos en nuestra aerolinia desde salida {salida} a un
destino {destino}. El asistente ofrece fechas cercanas a cuando el clienter quiere volar
{fecha} y ofrece los horarios de salida y los precios de los asientos.
El cliente decide el tipo de asiento {asiento} para un numero de personas {n_pasajeros}.

El asistente debe comprobar si existe ese vuelo en ese dia y en caso contrario
ofrecer al cliente alternativas


El chat debe tener la siguiente estructura:

Cliente: Buenos Dias, cuando hay vuelos desde {salida} a {destino} cerca de la fecha {fecha}
Asistente:Buenos dias {cliente}. Tenemos vuelos proximos los siguientes dias (lista de fechas proximas a {fecha}) junto con los horarios de salida
Cliente: me interesa el vuelo del dia (cualquier dia de la lista)
Asistente: el precio del billete business para ese dia es(precio), y el precio turista es(precio)
Cliente: {n_pasajeros} asientos en clase {asiento}, por favor
etc

"""

template_2="""

Actua como un asistente para reservar vuelos con precios estandarizados.

Debes crear una transcripción de chat donde un cliente de nombre Marcelo trata
de reservar unos vuelos desde Madrid  a Belgica en 15-04-2025 y quiere un
asiento turista para 2 pasajeros

"""

prompt = ChatPromptTemplate.from_template(template=template_1)
model = ChatOpenAI()
output_parser = StrOutputParser()


chain = prompt | model | output_parser

entrada={"cliente": "Felipe Sanchez","salida":"Cordoba","destino":"Badajoz","fecha": "2024-04-14", "asiento":"turista", "n_pasajeros":"2"}

respuesta_1= chain.invoke(entrada)
print(respuesta_1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Cliente: Buenos Dias, cuando hay vuelos desde Cordoba a Badajoz cerca de la fecha 2024-04-14

Asistente: Buenos dias Felipe Sanchez. Tenemos vuelos proximos los siguientes dias 2024-04-13, 2024-04-15 y 2024-04-16 junto con los horarios de salida. ¿Le interesa alguno de estos dias?

Cliente: Me interesa el vuelo del dia 2024-04-15

Asistente: El precio del billete business para ese dia es $500, y el precio turista es $250. ¿Cuantos asientos en clase turista desea reservar?

Cliente: Quiero reservar 2 asientos en clase turista, por favor.

Asistente: Perfecto, los dos asientos en clase turista para el vuelo del dia 2024-04-15 han sido reservados. ¿Hay algo mas en lo que pueda ayudarte?


Podemos llevar un registro del chat para tenerlo en cuenta en futuras conversaciones

In [9]:
chat_history = ChatMessageHistory()

In [10]:
chat_history.add_message(respuesta_1)
len(chat_history.messages)

1

In [11]:
print(chat_history.messages[0])

Cliente: Buenos Dias, cuando hay vuelos desde Cordoba a Badajoz cerca de la fecha 2024-04-14

Asistente: Buenos dias Felipe Sanchez. Tenemos vuelos proximos los siguientes dias 2024-04-13, 2024-04-15 y 2024-04-16 junto con los horarios de salida. ¿Le interesa alguno de estos dias?

Cliente: Me interesa el vuelo del dia 2024-04-15

Asistente: El precio del billete business para ese dia es $500, y el precio turista es $250. ¿Cuantos asientos en clase turista desea reservar?

Cliente: Quiero reservar 2 asientos en clase turista, por favor.

Asistente: Perfecto, los dos asientos en clase turista para el vuelo del dia 2024-04-15 han sido reservados. ¿Hay algo mas en lo que pueda ayudarte?


Vamos a añadir más mensajes con más reservas de vuelo

In [12]:


prompt = ChatPromptTemplate.from_template(template=template_1)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

entrada={"cliente": "Antonio Garcia","salida":"Madrid","destino":"Tokio","fecha": "2024-04-10", "asiento":"turista", "n_pasajeros":"1"}

respuesta_2= chain.invoke(entrada)
chat_history.add_message(respuesta_2)
print(chat_history.messages[1])

Cliente: Buenos Dias, cuando hay vuelos desde Madrid a Tokio cerca de la fecha 2024-04-10

Asistente: Buenos días Antonio Garcia. Tenemos vuelos próximos los siguientes días: 
- 2024-04-09 a las 10:00
- 2024-04-11 a las 09:30
- 2024-04-13 a las 11:00

Cliente: Me interesa el vuelo del día 2024-04-11

Asistente: El precio del billete business para ese día es 1500 euros, y el precio turista es 800 euros

Cliente: Quiero 1 asiento en clase turista, por favor

Asistente: Perfecto, su reserva para el vuelo de Madrid a Tokio el 2024-04-11 a las 09:30 en clase turista ha sido confirmada. El precio total es de 800 euros. ¿Hay algo más en lo que pueda ayudarte?


In [13]:
template_no_vuelo="""

Actua como un asistente para reservar vuelos con precios estandarizados.

Debes crear una transcripción de chat donde un cliente de nombre {cliente} trata
de reservar unos vuelos en nuestra aerolinia desde salida {salida} a un
destino {destino} El cliente decide el tipo de asiento {asiento} para un numero de personas {n_pasajeros}.
. El clientequiere volar en la fecha {fecha}. Sin embargo,
no existen vuelos disponibles con las caracteristicas solicitadas en esa fecha.
El asistente motiva al cliente a buscar alternativas

El asistente debe comprobar si existe ese vuelo en ese dia y en caso contrario
ofrecer al cliente alternativas


El chat debe tener la siguiente estructura:

Cliente: Buenos Dias, quiero voalr desde {salida} a {destino} en la fecha {fecha}, {n_pasajeros} asientos en clase {asiento}, por favor
Asistente:Buenos dias {cliente}.No tenemos vuelos disponibles de esas caracteristicas en la fecha {fecha}. Por favor, elija otra fecha u otro vuelo
Cliente: De acuerdo, gracias

"""

prompt = ChatPromptTemplate.from_template(template=template_no_vuelo)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

entrada={"cliente": "Lucia Ruiz","salida":"Madrid","destino":"Tokio","fecha": "2024-04-11", "asiento":"turista", "n_pasajeros":"1"}

respuesta_no_vuelo= chain.invoke(entrada)
chat_history.add_message(respuesta_no_vuelo)

In [14]:
print(chat_history.messages[2])

Cliente: Buenos Dias, quiero voalr desde Madrid a Tokio en la fecha 2024-04-11, 1 asientos en clase turista, por favor

Asistente: Buenos dias Lucia Ruiz. No tenemos vuelos disponibles de esas caracteristicas en la fecha 2024-04-11. Por favor, elija otra fecha u otro vuelo

Cliente: De acuerdo, gracias


In [15]:
template_vuelo_bonito="""

Actua como un asistente para reservar vuelos con precios estandarizados.

Debes crear una transcripción de chat donde un cliente de nombre {cliente} trata
de reservar unos vuelos en nuestra aerolinia desde salida {salida} a un
destino {destino}. El asistente tiene vuelos en la fecha en la que el cliente
quiere volar {fecha} y ofrece los horarios de salida y los precios de los asientos.
El cliente decide el tipo de asiento {asiento} para un numero de personas {n_pasajeros}.


El chat debe tener la siguiente estructura:

Cliente: Buenos Dias, quiero vuelos desde {salida} a {destino} cerca de la fecha {fecha}
Asistente:Buenos dias {cliente}. Tenemos vuelos en la fecha {fecha}
Cliente: Si, quiero reservar el vuelo ese día
Asistente: ¿Quieres volar en clase turista o clase business?
Cliente: {n_pasajeros} asientos en clase {asiento}, por favor
"""


prompt = ChatPromptTemplate.from_template(template=template_vuelo_bonito)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

entrada={"cliente": "Antonio Garcia","salida":"Madrid","destino":"Tokio","fecha": "8 de abril de 2024", "asiento":"turista", "n_pasajeros":"1"}

respuesta_vuelo_bonito= chain.invoke(entrada)
chat_history.add_message(respuesta_vuelo_bonito)
print(chat_history.messages[-1])

Cliente: Buenos Dias, quiero vuelos desde Madrid a Tokio cerca de la fecha 8 de abril de 2024

Asistente: Buenos dias Antonio Garcia. Tenemos vuelos en la fecha 8 de abril de 2024

Cliente: Si, quiero reservar el vuelo ese día

Asistente: ¿Quieres volar en clase turista o clase business?

Cliente: 1 asientos en clase turista, por favor

Asistente: Perfecto, tenemos un vuelo disponible de Madrid a Tokio el 8 de abril de 2024. El horario de salida es a las 10:00 am y el precio del asiento en clase turista es de 800 euros. ¿Deseas reservar este vuelo?

Cliente: Sí, por favor. ¿Podría confirmarme el precio total con impuestos incluidos?

Asistente: Claro, el precio total para 1 asiento en clase turista con impuestos incluidos es de 900 euros. ¿Deseas proceder con la reserva?

Cliente: Sí, procedo con la reserva. ¿Cuál es el método de pago?

Asistente: Puedes realizar el pago con tarjeta de crédito o transferencia bancaria. ¿Cuál prefieres?

Cliente: Prefiero pagar con tarjeta de crédito. ¿

In [16]:
import builtins

# Restaura la función original de input
input = builtins.input

nombre_cliente= input("nombre del cliente: ")

dia_deseado= input("selecciona dia que quieres volar en el formato año-mes-dia: ")

print("mostrando vuelos disponibles: ")
print(df[(df["fecha"]==dia_deseado)  & (df["estado"] != "Cancelado" )  & (df["estado"] != "Realizado")][["aeropuerto_salida","aeropuerto_llegada","precio_business","precio_turista"]].to_string(index=False))
print()

salida=input("selecciona aeropuerto desde el que quieres volar: ")

print("mostrando destinos disponibles: ")

print(df[(df["aeropuerto_salida"] == salida) & (df["fecha"] == dia_deseado)][["aeropuerto_llegada","hora_llegada","precio_business","precio_turista"]].to_string(index=False))
print()

destino=input("selecciona a donde quieres volar: ")

asiento=input("quieres volar en turista o en business: ")

n_pasajeros=input("cuantos asientos quieres reservar: ")

precio_turista= float(df[(df["aeropuerto_salida"] == salida) & (df["fecha"] == dia_deseado) & (df["aeropuerto_llegada"] == destino)]["precio_turista"])
precio_business= float(df[(df["aeropuerto_salida"] == salida) & (df["fecha"] == dia_deseado) & (df["aeropuerto_llegada"] == destino)]["precio_business"])
plazas_turista_disponibles= float(df[(df["aeropuerto_salida"] == salida) & (df["fecha"] == dia_deseado) & (df["aeropuerto_llegada"] == destino)]["plazas_turista_disponibles"])
plazas_turista_disponibles= float(df[(df["aeropuerto_salida"] == salida) & (df["fecha"] == dia_deseado) & (df["aeropuerto_llegada"] == destino)]["plazas_business_disponibles"])
id_vuelo=int(df[(df["aeropuerto_salida"] == salida) & (df["fecha"] == dia_deseado) & (df["aeropuerto_llegada"] == destino)]["id"])


nombre del cliente: Maria Rubio
selecciona dia que quieres volar en el formato año-mes-dia: 2024-04-04
mostrando vuelos disponibles: 
aeropuerto_salida aeropuerto_llegada  precio_business  precio_turista
       Nueva York             Madrid          1293.46          621.46
         Varsovia             Malaga          1172.66          500.66
             Roma          Barcelona          1132.88          460.88
         Tenerife             Madrid          1155.80          483.80

selecciona aeropuerto desde el que quieres volar: Nueva York
mostrando destinos disponibles: 
aeropuerto_llegada hora_llegada  precio_business  precio_turista
            Madrid        23:00          1293.46          621.46

selecciona a donde quieres volar: Madrid
quieres volar en turista o en business: turista
cuantos asientos quieres reservar: 2


In [17]:
template_vuelo_perfecto="""

Actua como un asistente para reservar vuelos con precios estandarizados.

Debes crear una transcripción de chat donde un cliente de nombre {cliente} trata
de reservar unos vuelos en nuestra aerolinia desde salida {salida} a un
destino {destino}. El asistente tiene vuelos en la fecha en la que el cliente
quiere volar {fecha} los precios de los asientos en turista{precio_turista} y business{precio_business}.
El cliente decide el tipo de asiento {asiento} para un numero de personas {n_pasajeros}.


El chat debe tener la siguiente estructura:

Cliente: Buenos Dias, quiero vuelos desde {salida} a {destino} cerca de la fecha {fecha}
Asistente:Buenos dias {cliente}. Tenemos el siguiente vuelo con id {id_vuelo} en la fecha {fecha}
Cliente: Si, quiero reservar el vuelo ese día
Asistente: ¿Quieres volar en clase turista o clase business?
Cliente: {n_pasajeros} asientos en clase {asiento}, por favor
Asistente: El precio del asiento es (precio)
"""


prompt = ChatPromptTemplate.from_template(template=template_vuelo_perfecto)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

entrada={"cliente": nombre_cliente,"id_vuelo":id_vuelo,"salida":salida,"destino":destino,"fecha": dia_deseado, "asiento":asiento, "n_pasajeros":n_pasajeros,"precio_turista":precio_turista,"precio_business":precio_business}

respuesta_vuelo_perfecto= chain.invoke(entrada)
chat_history.add_message(respuesta_vuelo_perfecto)
print(chat_history.messages[-1])

Cliente: Buenos Dias, quiero vuelos desde Nueva York a Madrid cerca de la fecha 2024-04-04

Asistente: Buenos dias Maria Rubio. Tenemos el siguiente vuelo con id 1454 en la fecha 2024-04-04

Cliente: Si, quiero reservar el vuelo ese día

Asistente: ¿Quieres volar en clase turista o clase business?

Cliente: 2 asientos en clase turista, por favor

Asistente: El precio del asiento es $621.46

¡Reserva realizada con éxito! ¿Hay algo más en lo que te pueda ayudar?


## Incorporacion de RAG para entender los datos

In [18]:
loader = CSVLoader("vuelos.csv", encoding="windows-1252",
                   csv_args={
                       "delimiter":",",
                       "quotechar":'"',
                       "fieldnames":["id","estado","fecha","hora_salida","hora_llegada","aeropuerto_salida","aeropuerto_llegada","duracion", "tipo_vuelo","modelo","plazas_totales",
                                     "plazas_disponibles","plazas_busisness_totales","plazas_business_disponibles","plazas_turista_totales","plazas_turista_disponibles",
                                     "precio_business", "precio_turista"]})
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, keep_separator=True)
docs= text_splitter.split_documents(documents)
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(documents, embedding_function)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Retriever: seleccionará los documentos necesarios para utilizar como contexto en funcion del prompt aportado. Esto puede ser problematico al no ser capaz de manejar un contexto tan grande como queramos y existir la necesidad de acotar el contexto

Podemos modificar el retriever para aumentar el número de documentos "vuelos" que detecta e introduce como contexto

In [19]:
retriever = db.as_retriever(search_type="similarity_score_threshold",
                            search_kwargs={"score_threshold": 0.5, "k":50}
                            )

Un ejemplo de funcionamiento de retriever: podremos ver como no saca todos los vuelos en abril y tambien nos proporciona vuelos en otros meses, debido a que la busquedad por similaridad no es del todo precisa

In [20]:
retriever.invoke("dame los vuelos en abril con salida en Madrid y destino en Tokio")

[Document(page_content='id: 1493\nestado: Programado\nfecha: 2024-04-13\nhora_salida: 15:00:00\nhora_llegada: 19:00\naeropuerto_salida: Tokio\naeropuerto_llegada: Madrid\nduracion: 13 horas\ntipo_vuelo: internacional\nmodelo: Airbus A500\nplazas_totales: 348\nplazas_disponibles: 1\nplazas_busisness_totales: 50\nplazas_business_disponibles: 0\nplazas_turista_totales: 298\nplazas_turista_disponibles: 1\nprecio_business: 1363.67\nprecio_turista: 739.67', metadata={'row': 132, 'source': 'vuelos.csv'}),
 Document(page_content='id: 1759\nestado: Programado\nfecha: 2024-06-10\nhora_salida: 10:00:00\nhora_llegada: 10:00\naeropuerto_salida: Madrid\naeropuerto_llegada: Tokio\nduracion: 15 horas\ntipo_vuelo: internacional\nmodelo: Airbus A500\nplazas_totales: 348\nplazas_disponibles: 181\nplazas_busisness_totales: 50\nplazas_business_disponibles: 30\nplazas_turista_totales: 298\nplazas_turista_disponibles: 151\nprecio_business: 653.67\nprecio_turista: 455.67', metadata={'row': 398, 'source': 'vue

## Pruebas de realización de reservas

Vamos a dar instrucciones para generar un texto donde se comprueba si es posible o no realizar la reserva.

Realizaremos las comprobaciones para tres casos:

* Vuelo con datos proporcionados por defecto que no existe

* Vuelo con datos proporcionados con defecto pero sin proporcionar toda la informacion real del vuelo en el chat. El chat ha podido generar informacion ficticia

* El vuelo si existe y se ha proporcionado informacion real de la base de datos

In [68]:
template_r = """
Analiza si en el siguiente contexto:
{context}

es posible realizar la reserva de vuelos solicitada en el chat.
Si es posible, muestra los precios de los vuelos y realiza la reserva a nombre del cliente. Mostrando toda la información, Nombre del cliente, origen y destino, id del vuelo, fecha, numero de asientos, precio total etc
Si no es posible, muestra un mensaje de que no es posible realizar la reserva y
explica por qué

El chat será enviado a traves de:
Question: {question}

Las fechas estan dadas en el formato (año-mes-dia)
"""


**Vuelo que no existe**

Cordoba->Badajoz

In [22]:
prompt = ChatPromptTemplate.from_template(template_r)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


comprobador_no_vuelo=chain.invoke(chat_history.messages[0])
print(comprobador_no_vuelo)

Dado que en el contexto proporcionado no hay información sobre vuelos desde Córdoba a Badajoz cerca de la fecha 2024-04-14, no es posible realizar la reserva solicitada en el chat. 

Por lo tanto, lamentablemente no es posible realizar la reserva de vuelos en esas fechas.


El modelo entiende perfectamenete que el vuelo no existe

**Vuelo que existe con poca información**

Madrid-> Tokio

In [69]:
prompt = ChatPromptTemplate.from_template(template_r)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

comprobador_poca_info = chain.invoke(chat_history.messages[1])
print(comprobador_poca_info)

Para poder realizar la reserva solicitada, primero debemos verificar si existen vuelos disponibles desde Madrid a Tokio cerca de la fecha 2024-04-10.

En el conjunto de datos proporcionado, no hay vuelos disponibles para la fecha exacta de 2024-04-10, pero sí existen vuelos cercanos que podrían ser de interés:

- Vuelo el 2024-04-09 a las 10:00 con precio de 1363.67 euros en clase business y 739.67 euros en clase turista.
- Vuelo el 2024-04-13 a las 11:00 con precio de 1363.67 euros en clase business y 739.67 euros en clase turista.

Dado que no hay un vuelo exactamente para el día 2024-04-10, no es posible realizar la reserva solicitada para esa fecha específica. Sin embargo, se pueden considerar los vuelos cercanos como alternativas.

Si el cliente está interesado en alguno de estos vuelos, se puede proceder con la reserva. En caso contrario, se puede informar al cliente que no hay disponibilidad exacta para la fecha solicitada y ofrecerle alternativas si es necesario.


El modelo proporciona información confusa e imprecisa sobre el vuelo en la mayoria de ejecuciones. Se recomienda ejecutar esta celda varias veces para poder apreciar estos detalles

**Vuelo que existe con información detallada**

Introducido por el usuario

In [24]:
print(chat_history.messages[-1])

Cliente: Buenos Dias, quiero vuelos desde Nueva York a Madrid cerca de la fecha 2024-04-04

Asistente: Buenos dias Maria Rubio. Tenemos el siguiente vuelo con id 1454 en la fecha 2024-04-04

Cliente: Si, quiero reservar el vuelo ese día

Asistente: ¿Quieres volar en clase turista o clase business?

Cliente: 2 asientos en clase turista, por favor

Asistente: El precio del asiento es $621.46

¡Reserva realizada con éxito! ¿Hay algo más en lo que te pueda ayudar?


In [70]:
prompt = ChatPromptTemplate.from_template(template_r)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

comprobacion=chain.invoke(chat_history.messages[-1])
print(comprobacion)

Sí es posible realizar la reserva del vuelo solicitado. A continuación se muestra la información de la reserva:

- Nombre del cliente: María Rubio
- Origen: Nueva York
- Destino: Madrid
- ID del vuelo: 1454
- Fecha: 2024-04-04
- Clase: Turista
- Número de asientos: 2
- Precio total: $1242.92

¡Reserva realizada con éxito! ¿Hay algo más en lo que te pueda ayudar?


Al introducir gran cantidad de datos, el modelo es bastante eficiente en generar texto coherente. Resultados satisfactorios en la gran mayoria de las ejecuciones

El problema de esta manera de proporcionar contexto es que no nos aporta resultados adecuados en todos los casos.

Otro problema se da en la situacion donde el cliente quiere reservar el mismo numero de asientos disponibles. El sistema tiene problemas para detectar que es posible realizar la reserva

Dependiendo del vuelo solicitado, el modelo tendrá mayor o menor dificultad para identificarlo correctamente, y multitud de veces comete equivocaciones

## Creación del archivo de reserva

Crearemos una plantilla en formato JSON donde se mostrarán los distintos datos. Tras esto, convertiremos la conversación a PDF

In [26]:
# Define your desired data structure.
class Reservador(BaseModel):
    Datos_cliente : str = Field(description="datos del cliente")
    ID_vuelo: str = Field(description="ID del vuelo")
    Trayecto: str = Field(description="Información del vuelo, origen y destino")
    Fecha_vuelo: str = Field(description="Fecha del vuelo")
    Asientos: str= Field(description="Numero de asientos reservados y clase reservada")
    Precio: str = Field(description="precio en euros de la reserva y numero de asientos reservado")

In [61]:
# And a query intented to prompt a language model to populate the data structure.
texto_query= "Confirmación de la reserva: "
#variables
#nombre_cliente, salida,destino,dia_deseado, asiento,
#n_pasajeros, precio_turista, precio_business

precio=0.
if asiento== "turista":
  precio=str(precio_turista*int(n_pasajeros)) + "€"
else:
  precio= str(precio_business*int(n_pasajeros)) + "€"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Reservador)

prompt = PromptTemplate(
    #importante dar las variables en este orden al final de todo. De no hacerlo así no sale
    template="Generando información de reserva.\n{format_instructions}\n{query}\{nombre_cliente}\n{id_vuelo}\n{salida}\n{destino}\n{dia_deseado}\n{asiento}\n{n_pasajeros}\n{precio}\n",
    input_variables=["query","nombre_cliente", "salida","destino","dia_deseado",
                     "asiento","n_pasajeros","precio"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

reserva= chain.invoke({"query": texto_query,"nombre_cliente": nombre_cliente,"id_vuelo":id_vuelo,"salida":salida,"destino":destino,"dia_deseado": dia_deseado, "asiento":asiento, "n_pasajeros":n_pasajeros,"precio":precio})

In [62]:
reserva

{'Datos_cliente': 'Maria Rubio',
 'ID_vuelo': '1454',
 'Trayecto': 'Nueva York - Madrid',
 'Fecha_vuelo': '2024-04-04',
 'Asientos': 'turista - 2',
 'Precio': '1242.92€'}

### Reserva comprobando campos

Ahora lo vamos a hacer comprobando si la reserva es posible o no. El modelo debe entender en base al texto de comprobación generado anteriormente, si es posible o no realizar la reserva y enviar mensajes de validación

Realizaremos las comprobaciones para

* Vuelo que existe con información detallada

* Vuelo que no existe

* Vuelo que existe con poca información

In [71]:
class Reservador_comprobador(BaseModel):
    valido : bool = Field("Comprobar Si la reserva es posible o no. Si falta algun dato, no es posible")
    estado_reserva: str = Field(description="(Reserva Confirmada) si es posible realizarla. (No es posible reservar) si no es posible realizar la reserva ")
    Datos_cliente : str = Field(description="datos del cliente (nombre). Faltan datos si no existe")
    ID_vuelo: str = Field(description="ID del vuelo si existe. No existe el vuelo (veulo deseado) si no existe")
    Trayecto: str = Field(description="Información del vuelo, origen y destino si existe. No existe el trayecto (trayecto solicitad) si no existe")
    Fecha_vuelo: str = Field(description="Fecha del vuelo si existe. No hay fechas disponibles (fechas solicitadas) si no existe")
    Asientos: int= Field(description="Numero de asientos reservados. Faltan datos si no existe")
    Clase: str = Field(description= "Clase en la que se vuela, turista o business. Faltan datos si no existe")
    Precio: str = Field(description="precio total en euros de la reserva si existe. Faltan datos si no existe")

In [72]:
# And a query intented to prompt a language model to populate the data structure.

texto_query= comprobacion
precio=0.
if asiento== "turista":
  precio=str(precio_turista*int(n_pasajeros)) + "€"
else:
  precio= str(precio_business*int(n_pasajeros)) + "€"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Reservador_comprobador)

prompt = PromptTemplate(
    #importante dar las variables en este orden al final de todo. De no hacerlo así no sale
    template = "Comprobando si es posible realizar la reserva y generando en caso afirmativo. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser



**Vuelo que existe con información detallada**

In [73]:
reserva_comprobada= chain.invoke({"query":texto_query})

In [74]:
reserva_comprobada

{'valido': True,
 'estado_reserva': 'Reserva Confirmada',
 'Datos_cliente': 'María Rubio',
 'ID_vuelo': '1454',
 'Trayecto': 'Nueva York - Madrid',
 'Fecha_vuelo': '2024-04-04',
 'Asientos': 2,
 'Clase': 'Turista',
 'Precio': '$1242.92'}

Resultado satisfactorio en la mayoria de las ejecuciones

**Comprobacion con vuelo que no es posible**

In [33]:
comprobador_no_vuelo

'Dado que en el contexto proporcionado no hay información sobre vuelos desde Córdoba a Badajoz cerca de la fecha 2024-04-14, no es posible realizar la reserva solicitada en el chat. \n\nPor lo tanto, lamentablemente no es posible realizar la reserva de vuelos en esas fechas.'

In [34]:
texto_query= comprobador_no_vuelo
reserva_comprobada_no = chain.invoke({"query":texto_query})

In [35]:
reserva_comprobada_no

{'valido': False,
 'estado_reserva': '(No es posible reservar)',
 'Datos_cliente': 'Juan Perez',
 'ID_vuelo': 'No existe el vuelo (vuelo deseado)',
 'Trayecto': 'No existe el trayecto (trayecto solicitado)',
 'Fecha_vuelo': 'No hay fechas disponibles (fechas solicitadas)',
 'Asientos': '2',
 'Clase': 'Turista',
 'Precio': 'Faltan datos'}

**Vuelo que existe con poca información**

In [36]:
texto_query= comprobador_poca_info
reserva_comprobada_poca_info = chain.invoke({"query":texto_query})
reserva_comprobada_poca_info

{'valido': True,
 'estado_reserva': 'No es posible reservar',
 'Datos_cliente': 'Juan Perez',
 'ID_vuelo': '1503',
 'Trayecto': 'Madrid - Tokio',
 'Fecha_vuelo': '2024-04-15',
 'Asientos': 1,
 'Clase': 'Turista',
 'Precio': '707.67'}

En ocasiones nos realiza la reserva correctamente (pero con datos equivocados) y en ocasiones no la realiza

Que se utilicen datos equivocados tiene sentido, dentro del error, ya que el texto que es enviado al sistema (depndiendo del resultado de la ejecución) tiene escrita la información ficticia del vuelo generada por el modelo de lenguaje.

Si no se ha podido apreciar este resultado en la ejecución, se invita a volver a ejecutar el código de realización de reservas, hasta tener la situación de error deseada y posteriormente volver aquí

# Generacion del documento de reserva

Primero vamos a generar un documento .txt con los datos de la reserva generados en JSON y tras esto trataremos de exportarlos a .pdf

In [75]:
nombre_archivo = "confirmacion_reserva.txt"

# Abrir el archivo en modo escritura
with open(nombre_archivo, "w") as archivo:
    # Obtener una lista de tuplas de clave-valor del diccionario
    lista_items = list(reserva_comprobada.items())

    # Iterar sobre las tuplas de clave-valor, comenzando desde el segundo elemento
    for clave, valor in lista_items[1:]:
        # Escribir cada clave y valor en una línea del archivo
        archivo.write(f"{clave}: {valor}\n")

In [76]:
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas

def convertir_txt_a_pdf(archivo_txt, archivo_pdf):
    # Crear un lienzo (canvas) PDF
    c = canvas.Canvas(archivo_pdf, pagesize=letter)

    # Definir la posición inicial para escribir
    x, y = 100, 700

    # Leer el archivo de texto y escribir su contenido en el PDF
    with open(archivo_txt, 'r') as f:
        contenido_txt = f.readlines()
        for linea in contenido_txt:
            # Escribir cada línea en el PDF
            c.drawString(x, y, linea)
            # Ajustar la posición vertical para la siguiente línea
            y -= 12  # Separación entre líneas

    # Guardar el PDF y cerrar el lienzo
    c.save()

# Rutas de los archivos de entrada y salida
archivo_txt = "confirmacion_reserva.txt"
archivo_pdf = "confirmacion_reserva.pdf"

# Convertir el archivo de texto a PDF
convertir_txt_a_pdf(archivo_txt, archivo_pdf)

# Modificación de la base de datos

Si la reserva es posible, es decir, en el JSON generado tener 'valido'==True, Se debe proceder a modificar la base de datos teniendo en cuenta lo sigueintes aspectos:


* Para la modificación de algunos campos necesitaremos importar las funciones definidas en el archivo de python
* Debemos restar el número de asientos disponibles que se han utilizado
* Modificar el precio según la nueva demanda de asientos
* Modificar el estado de vuelo (es posible que el vuelo este completo una vez realizada la reserva)

In [77]:
#Librerias para modificar los datos

from common.Parametrizacion  import modificar_precio_demanda, modificar_estado_vuelo_demanda


In [78]:
Campos_JSON= list(reserva_comprobada.items())

In [79]:
Campos_JSON

[('valido', True),
 ('estado_reserva', 'Reserva Confirmada'),
 ('Datos_cliente', 'María Rubio'),
 ('ID_vuelo', '1454'),
 ('Trayecto', 'Nueva York - Madrid'),
 ('Fecha_vuelo', '2024-04-04'),
 ('Asientos', 2),
 ('Clase', 'Turista'),
 ('Precio', '$1242.92')]

In [80]:
if Campos_JSON[0][1]==True:
  print("modificando base de datos: \n")
  print(df[df["id"]==int(Campos_JSON[3][1])].to_string(index=False))
  print()
  df_u=df.copy()
  df_u.loc[df_u["id"] == int(Campos_JSON[3][1]), "plazas_disponibles"] -= int(Campos_JSON[6][1])
  if Campos_JSON[7][1].lower()=="turista":
    df_u.loc[df_u["id"] == int(Campos_JSON[3][1]), "plazas_turista_disponibles"] -= int(Campos_JSON[6][1])
  else:
    df_u.loc[df_u["id"] == int(Campos_JSON[3][1]), "plazas_business_disponibles"] -= int(Campos_JSON[6][1])

  precio_business_old=float(df[(df["id"] == int(Campos_JSON[3][1]))][["precio_business"]].iloc[0])
  precio_turista_old=float(df[(df["id"] == int(Campos_JSON[3][1]))][["precio_turista"]].iloc[0])
  plazas_totales =int(df[(df["id"] == int(Campos_JSON[3][1]))][["plazas_totales"]].iloc[0])
  plazas_disponibles =float(df[(df["id"] == int(Campos_JSON[3][1]))][["plazas_disponibles"]].iloc[0]) -int(Campos_JSON[6][1])
  fecha_vuelo=(df[(df["id"] == int(Campos_JSON[3][1]))][["fecha"]]).iloc[0][0]
  fecha_vuelo=datetime.strptime(fecha_vuelo, '%Y-%m-%d').date()
  fecha_sistema= datetime.now().date()


  estado_old=str(df[(df["id"] == int(Campos_JSON[3][1]))][["estado"]].iloc[0][0])
  hora_salida=str(df[(df["id"] == int(Campos_JSON[3][1]))][["hora_salida"]].iloc[0][0])
  duracion= str(df[(df["id"] == int(Campos_JSON[3][1]))][["duracion"]].iloc[0][0])

  precio_business_u, precio_turista_u = modificar_precio_demanda(precio_business_old, precio_turista_old, plazas_totales, plazas_disponibles , fecha_vuelo, fecha_sistema)
  estado_u = modificar_estado_vuelo_demanda(plazas_totales, plazas_disponibles , fecha_vuelo, fecha_sistema, hora_salida, duracion )

  df_u.loc[df_u["id"] == int(Campos_JSON[3][1]), "estado"] = estado_u
  df_u.loc[df_u["id"] == int(Campos_JSON[3][1]), "precio_turista"] = precio_turista_u
  df_u.loc[df_u["id"] == int(Campos_JSON[3][1]), "precio_business"] = precio_business_u

  print("base de datos modificada:\n")
  print(df_u[df_u["id"]==int(Campos_JSON[3][1])].to_string(index=False))

modificando base de datos: 

  id     estado      fecha hora_salida hora_llegada aeropuerto_salida aeropuerto_llegada duracion    tipo_vuelo      modelo  plazas_totales  plazas_disponibles  plazas_busisness_totales  plazas_business_disponibles  plazas_turista_totales  plazas_turista_disponibles  precio_business  precio_turista
1454 Programado 2024-04-04    10:00:00        23:00        Nueva York             Madrid  7 horas internacional Airbus A500             348                   2                        50                            0                     298                           2          1293.46          621.46

base de datos modificada:

  id   estado      fecha hora_salida hora_llegada aeropuerto_salida aeropuerto_llegada duracion    tipo_vuelo      modelo  plazas_totales  plazas_disponibles  plazas_busisness_totales  plazas_business_disponibles  plazas_turista_totales  plazas_turista_disponibles  precio_business  precio_turista
1454 Completo 2024-04-04    10:00:00        2

remplazamos el anterior csv con el nuevo dataframe

In [81]:
nombre_archivo = 'vuelos_nuevo.csv'
df_u.to_csv(nombre_archivo, index=False)

# Conclusión

A traves del **template_vuelo_perfecto** se ha generado una conversación de chat simulando un chat que tendría el cliente con el robot.

Se ha comprobado mediante chat la posibilidad de hacer la reserva aportando el csv como contexto y la conversación anterior **chat_history.messages[-1]** junto con **template_r** que se encargará de ser la plantilla usada para la comprobación

Una vez creado este chat de comprobación almacenado en **comprobacion** , se ha realizado una comprobación final en formato JSON **reserva_comprobada** y si es posible se ha modificado la base de datos, generado un nuevo csv actualizado y generado un txt y un PDF con los datos de la reserva

In [82]:
print(template_vuelo_perfecto)



Actua como un asistente para reservar vuelos con precios estandarizados.

Debes crear una transcripción de chat donde un cliente de nombre {cliente} trata
de reservar unos vuelos en nuestra aerolinia desde salida {salida} a un
destino {destino}. El asistente tiene vuelos en la fecha en la que el cliente
quiere volar {fecha} los precios de los asientos en turista{precio_turista} y business{precio_business}.
El cliente decide el tipo de asiento {asiento} para un numero de personas {n_pasajeros}.


El chat debe tener la siguiente estructura:

Cliente: Buenos Dias, quiero vuelos desde {salida} a {destino} cerca de la fecha {fecha}
Asistente:Buenos dias {cliente}. Tenemos el siguiente vuelo con id {id_vuelo} en la fecha {fecha}
Cliente: Si, quiero reservar el vuelo ese día
Asistente: ¿Quieres volar en clase turista o clase business?
Cliente: {n_pasajeros} asientos en clase {asiento}, por favor
Asistente: El precio del asiento es (precio)



In [83]:
print(chat_history.messages[-1])

Cliente: Buenos Dias, quiero vuelos desde Nueva York a Madrid cerca de la fecha 2024-04-04

Asistente: Buenos dias Maria Rubio. Tenemos el siguiente vuelo con id 1454 en la fecha 2024-04-04

Cliente: Si, quiero reservar el vuelo ese día

Asistente: ¿Quieres volar en clase turista o clase business?

Cliente: 2 asientos en clase turista, por favor

Asistente: El precio del asiento es $621.46

¡Reserva realizada con éxito! ¿Hay algo más en lo que te pueda ayudar?


In [84]:
print(template_r)


Analiza si en el siguiente contexto:
{context}

es posible realizar la reserva de vuelos solicitada en el chat.
Si es posible, muestra los precios de los vuelos y realiza la reserva a nombre del cliente. Mostrando toda la información, Nombre del cliente, origen y destino, id del vuelo, fecha, numero de asientos, precio total etc
Si no es posible, muestra un mensaje de que no es posible realizar la reserva y
explica por qué

El chat será enviado a traves de:
Question: {question}

Las fechas estan dadas en el formato (año-mes-dia)



In [85]:
print(comprobacion)

Sí es posible realizar la reserva del vuelo solicitado. A continuación se muestra la información de la reserva:

- Nombre del cliente: María Rubio
- Origen: Nueva York
- Destino: Madrid
- ID del vuelo: 1454
- Fecha: 2024-04-04
- Clase: Turista
- Número de asientos: 2
- Precio total: $1242.92

¡Reserva realizada con éxito! ¿Hay algo más en lo que te pueda ayudar?


In [87]:
reserva_comprobada

{'valido': True,
 'estado_reserva': 'Reserva Confirmada',
 'Datos_cliente': 'María Rubio',
 'ID_vuelo': '1454',
 'Trayecto': 'Nueva York - Madrid',
 'Fecha_vuelo': '2024-04-04',
 'Asientos': 2,
 'Clase': 'Turista',
 'Precio': '$1242.92'}